# 5.6. GPU

In [1]:
!nvidia-smi

Sun Apr 16 16:22:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 45%   36C    P8    N/A /  75W |    137MiB /  4096MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 5.6.1. 计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i = 0): #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [5]:
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 5.6.2. 张量与GPU

In [6]:
# 查询张量所在设备
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

5.6.2.1. 存储在GPU上

In [7]:
# 在GPU上存储张量:可以在创建张量时指定存储设备
X = torch.ones((2, 10), device=try_gpu())
X

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0')

In [9]:
# 实际只有一个gpu，会用cpu
Y = torch.rand(2, 10, device=try_gpu(1))
Y

tensor([[0.7026, 0.9213, 0.0069, 0.4451, 0.8105, 0.0278, 0.4956, 0.7902, 0.0353,
         0.8491],
        [0.1729, 0.7375, 0.6179, 0.3915, 0.7305, 0.4713, 0.9736, 0.7276, 0.4268,
         0.2533]])

5.6.2.2. 复制

In [10]:
# operating defferent data from defferent devices should copy one to other device
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.7026, 0.9213, 0.0069, 0.4451, 0.8105, 0.0278, 0.4956, 0.7902, 0.0353,
         0.8491],
        [0.1729, 0.7375, 0.6179, 0.3915, 0.7305, 0.4713, 0.9736, 0.7276, 0.4268,
         0.2533]])
tensor([[0.7026, 0.9213, 0.0069, 0.4451, 0.8105, 0.0278, 0.4956, 0.7902, 0.0353,
         0.8491],
        [0.1729, 0.7375, 0.6179, 0.3915, 0.7305, 0.4713, 0.9736, 0.7276, 0.4268,
         0.2533]], device='cuda:0')


In [11]:
X + Z

tensor([[1.7026, 1.9213, 1.0069, 1.4451, 1.8105, 1.0278, 1.4956, 1.7902, 1.0353,
         1.8491],
        [1.1729, 1.7375, 1.6179, 1.3915, 1.7305, 1.4713, 1.9736, 1.7276, 1.4268,
         1.2533]], device='cuda:0')

In [12]:
# 变量已经存在在某设备，再复制到该设备不重新分配空间
Z.cuda(0) is Z

True

5.6.2.3. 旁注

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。 但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。 这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收）， 然后才能继续进行更多的操作。 这就是为什么拷贝操作要格外小心。 根据经验，多个小操作比一个大操作糟糕得多。 此外，一次执行几个操作比代码中散布的许多单个操作要好得多。 如果一个设备必须等待另一个设备才能执行其他操作， 那么这样的操作可能会阻塞。 这有点像排队订购咖啡，而不像通过电话预先订购： 当客人到店的时候，咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时， 如果数据不在内存中，框架会首先将其复制到内存中， 这会导致额外的传输开销。 更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

# 5.6.3. 神经网络与GPU

In [13]:
# 神经网络模型可以指定设备
net = nn.Sequential(nn.Linear(10, 1))
net = net.to(device=try_gpu())

In [14]:
net(X)

tensor([[-0.4201],
        [-0.4201]], device='cuda:0', grad_fn=<AddmmBackward>)

In [15]:
net[0].weight.data.device

device(type='cuda', index=0)

# 5.6.4. 小结

我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。

深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。

不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。

# 5.6.5. 练习

1.尝试一个计算量更大的任务，比如大矩阵的乘法，看看CPU和GPU之间的速度差异。再试一个计算量很小的任务呢？
大矩阵乘法gpu快很多
小矩阵乘法cpu反而快一些

In [18]:
X = torch.randn((1000, 1000), device=try_gpu())
Y = torch.randn((1000, 1000), device=try_gpu())
M = X.cpu()
N = Y.cpu()
print(X)
print(M)

tensor([[ 0.7713,  0.0943,  0.2027,  ..., -0.3198,  0.1043, -1.0616],
        [-0.5781, -2.0439,  0.2811,  ...,  2.0557,  0.3688, -0.4359],
        [-0.9240, -0.0954, -2.1521,  ...,  1.7676, -0.2314, -1.7119],
        ...,
        [-0.8144,  1.4072, -0.5928,  ...,  0.4605, -0.7303,  1.5346],
        [ 0.9191, -1.0771, -0.6726,  ..., -1.1343,  0.3618, -0.3338],
        [-2.1029, -0.8234,  2.1010,  ..., -1.4013, -0.5891,  1.6257]],
       device='cuda:0')
tensor([[ 0.7713,  0.0943,  0.2027,  ..., -0.3198,  0.1043, -1.0616],
        [-0.5781, -2.0439,  0.2811,  ...,  2.0557,  0.3688, -0.4359],
        [-0.9240, -0.0954, -2.1521,  ...,  1.7676, -0.2314, -1.7119],
        ...,
        [-0.8144,  1.4072, -0.5928,  ...,  0.4605, -0.7303,  1.5346],
        [ 0.9191, -1.0771, -0.6726,  ..., -1.1343,  0.3618, -0.3338],
        [-2.1029, -0.8234,  2.1010,  ..., -1.4013, -0.5891,  1.6257]])


In [22]:
from d2l import torch as d2l

timer = d2l.Timer()
for i in range(10):
    X = torch.mm(X, Y)
print(f'gpu multiply 10 times need {timer.stop():.2f} sec')

timer = d2l.Timer()
for i in range(10):
    M = torch.mm(M, N)
print(f'cpu multiply 10 times need {timer.stop():.2f} sec')

gpu multiply 10 times need 0.00 sec
cpu multiply 10 times need 0.24 sec


In [27]:
X = torch.randn((10, 10), device=try_gpu())
Y = torch.randn((10, 10), device=try_gpu())
M = X.cpu()
N = Y.cpu()

timer = d2l.Timer()
for i in range(10000):
    X = torch.mm(X, Y)
print(f'gpu multiply 10 times need {timer.stop():.2f} sec')

timer = d2l.Timer()
for i in range(10000):
    M = torch.mm(M, N)
print(f'cpu multiply 10 times need {timer.stop():.2f} sec')

gpu multiply 10 times need 0.07 sec
cpu multiply 10 times need 0.03 sec
